<a href="https://colab.research.google.com/github/lawlete/Varios/blob/main/Procesamiento_de_Lenguaje_Natural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Procesamiento de Lenguaje Natural (NLP)**

Utiliza Python y una librería como Hugging Face Transformers para entrenar un modelo que realice una tarea de clasificación de texto (por ejemplo, clasificación de opiniones: positiva o negativa). Proporciona un conjunto de datos de ejemplo (puede ser un pequeño conjunto de datos de reseñas) y demuestra cómo entrenar el modelo, evaluarlo y ajustarlo.

Paso 1: Instalación de dependencias
Primero, instala las librerías necesarias

In [5]:
!pip install python-dotenv
!pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**Cargamos la API TOKEN de Hugging Face**

In [13]:
from dotenv import load_dotenv, find_dotenv
import os
# Verifica y carga el file con las variables de entorno si existe
if load_dotenv(find_dotenv()):
    if "HF_TOKEN" in os.environ: # Verifica que existe la variable HF_KEY
        print("HUGGING FACE API TOKEN cargada") # Imprime el mensaje Si existe la variable de OpenAI
    else:
        print("No se cargo la api_key de HUGGING FACE, debera cargarla manualmente")
else:
      print("No se cargo la api_key, debera cargarla manualmente")

HUGGING FACE API TOKEN cargada


Paso 2: Cargar un conjunto de datos
Utilizaremos el conjunto de datos IMDB, que contiene reseñas de películas etiquetadas como positivas (1) o negativas (0). Este conjunto de datos está disponible en la librería datasets de Hugging Face.

In [14]:
from datasets import load_dataset

# Cargar el conjunto de datos IMDB
dataset = load_dataset("imdb")

# Ver la estructura del dataset
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


**Paso 3: Preprocesamiento de los datos
Tokenizamos el texto utilizando un tokenizador preentrenado de Hugging Face.

En este caso, usaremos el modelo distilbert-base-uncased.**

In [15]:
from transformers import AutoTokenizer

# Cargar el tokenizador
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Función para tokenizar el texto
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

# Aplicar tokenización al dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

**Paso 4: Dividir el conjunto de datos
Dividimos el conjunto de datos en entrenamiento y evaluación.**

In [16]:
# Dividir el dataset en entrenamiento y evaluación
train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))  # Usamos solo 1000 muestras para entrenamiento rápido
eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(100))    # Usamos 100 muestras para evaluación

**Paso 5: Entrenar el modelo**

**Cargamos un modelo preentrenado y lo ajustamos para la tarea de clasificación.**

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Cargar el modelo preentrenado
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Crear el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Entrenar el modelo
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alfredolawler (alfredolawler-lawer-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss


**Paso 6: Evaluar el modelo**

**Evaluamos el modelo en el conjunto de evaluación.**

In [ ]:
# Evaluar el modelo
eval_results = trainer.evaluate()
print(f"Precisión: {eval_results['eval_accuracy']}")

**Paso 7: Realizar predicciones**

**Podemos usar el modelo entrenado para clasificar nuevas reseñas.**  

In [ ]:
# Función para predecir la clase de un texto
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(dim=-1)
    return "Positiva" if predictions == 1 else "Negativa"

# Probar con una reseña de ejemplo
review = "This movie was fantastic! I loved every minute of it."
print(f"Predicción: {predict_sentiment(review)}")

**Paso 8: Ajustar el modelo (opcional)**

Si el rendimiento no es satisfactorio, es posible ajustar los hiperparámetros (por ejemplo, learning_rate, num_train_epochs, batch_size) o probar con otro modelo preentrenado.

**Conjunto de datos de ejemplo mas reducido**

Si se prefiere usar un conjunto de datos más pequeño, se puedes crear uno manualmente:

In [ ]:
import pandas as pd

# Crear un pequeño conjunto de datos de ejemplo
data = {
    "text": [
        "I loved this movie, it was amazing!",
        "Terrible film, I hated it.",
        "The acting was great, but the plot was boring.",
        "Absolutely fantastic!",
        "Worst movie ever."
    ],
    "label": [1, 0, 0, 1, 0]  # 1 = Positiva, 0 = Negativa
}

df = pd.DataFrame(data)
print(df)